In [1]:
%load_ext autoreload
%autoreload 

In [74]:
from dotenv import load_dotenv
load_dotenv() 
from names import DATASET_ID, MODEL_ID
from data import load_db
import os
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.vectorstores import DeepLake
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

embeddings = OpenAIEmbeddings(model=MODEL_ID)
dataset_path = f"hub://{os.environ['ACTIVELOOP_ORG_ID']}/{DATASET_ID}"

db = load_db(dataset_path, embedding_function=embeddings, token=os.environ['ACTIVELOOP_TOKEN'], org_id=os.environ["ACTIVELOOP_ORG_ID"], read_only=True)

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/zuppif/disney-lyrics-emotions



-

hub://zuppif/disney-lyrics-emotions loaded successfully.

Deep Lake Dataset in hub://zuppif/disney-lyrics-emotions already exists, loading from the storage
Dataset(path='hub://zuppif/disney-lyrics-emotions', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape      dtype  compression
  -------   -------   -------    -------  ------- 
 embedding  generic  (85, 1536)  float32   None   
    ids      text     (85, 1)      str     None   
 metadata    json     (85, 1)      str     None   
   text      text     (85, 1)      str     None   


## Using similarity search

In [75]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from pathlib import Path

prompt = PromptTemplate(
    input_variables=["content"],
    template=Path("prompts/bot.prompt").read_text(),
)

llm = ChatOpenAI(temperature=0.7)

chain = LLMChain(llm=llm, prompt=prompt)

In [76]:
emotions = chain.run(content="Damn I am feeling so tired")
emotions

'Exhaustion, Fatigue, Sleepiness, Drained.'

In [77]:
matches = db.similarity_search_with_score(emotions, distance_metric="cos")
print(matches)
doc, score = matches[0]
print(doc.metadata["embed_url"])
print(doc)

from IPython.display import IFrame
IFrame(doc.metadata["embed_url"], width=700, height=350)

[(Document(page_content='Hopeful, determined, inspired, optimistic, longing, driven, passionate, adventurous.', metadata={'movie': 'Hercules', 'name': 'Go the Distance', 'embed_url': 'https://open.spotify.com/embed/track/0D1OY0M5A0qD5HGBvFmFid?utm_source=generator'}), 0.8135085701942444), (Document(page_content='upset, mad, regret, sad, fine, longing, hopeful, impatient', metadata={'movie': 'Encanto', 'name': 'Waiting on a Miracle', 'embed_url': 'https://open.spotify.com/embed/track/3oRW9ZGPRbLRMneQ5lwflt?utm_source=generator'}), 0.8108540177345276), (Document(page_content='nasty, repentant, magic, sad, lonely, bored, withdrawn, busy', metadata={'movie': 'The Little Mermaid', 'name': 'Poor Unfortunate Souls', 'embed_url': 'https://open.spotify.com/embed/track/7zsw78LtXUD7JfEwH64HK2?utm_source=generator'}), 0.8080281615257263), (Document(page_content='hopeful, optimistic, dreamy, inspired, happy, content, fulfilled, grateful', metadata={'movie': 'Pinocchio', 'name': 'When You Wish Upon 

## Using all the songs emotions in the prommpt

In [295]:
import json

prompt = PromptTemplate(
    input_variables=["songs", "user_input"],
    template=Path("prompts/bot_with_summary.prompt").read_text(),
)

llm = ChatOpenAI(temperature=1)

chain = LLMChain(llm=llm, prompt=prompt)

Let's create the songs string

In [296]:
with open("data/emotions_with_spotify_url.json", "r") as f:
    data = json.load(f)
    
movies_and_names_to_songs = {}

In [297]:
songs_str = ""

for movie, songs in data.items():
    for song in songs:
        movie_and_name = f"{movie};{song['name']}".lower()
        songs_str += f"{movie_and_name}:{song['text']}\n"
        movies_and_names_to_songs[movie_and_name] = song

In [301]:
songs_str

"aladdin;friend like me:excitement, happiness, humor, friendliness, generosity, willingness, eagerness, confidence\naladdin;arabian nights:nostalgic, adventurous, exotic, intense, romantic, mysterious, whimsical, passionate\naladdin;a whole new world:wonder, joy, excitement, adventure, romance, freedom, happiness, awe\naladdin;one jump ahead:Thrill, fear, desperation, humor, defiance, friendship, hunger, rebellion.\naladdin (live-action);a whole new world:nostalgia, wonder, excitement, joy, romance, adventure, hope, inspiration\naladdin (live-action);a whole new world (end title):joy, wonder, excitement, adventure, love, freedom, exhilaration, enchantment\naladdin (live-action);friend like me:excitement, confidence, generosity, humor, friendliness, helpfulness, empowerment, joyfulness\naladdin (live-action);one jump ahead:excitement, fear, desperation, humor, defiance, loneliness, camaraderie, determination\naladdin (live-action);speechless (full):determination, empowerment, resilience

In [298]:
# movies_and_names_to_songs

In [299]:
# prompt.format(songs=songs_str, user_input="I am feeling great today")

In [300]:
res = chain.run(songs=songs_str, user_input="I need energy").lower()

In [ ]:
print(res)
doc = movies_and_names_to_songs[res]

from IPython.display import IFrame
IFrame(doc["embed_url"], width=700, height=350)